<a href="https://colab.research.google.com/github/Znerf/diabetes-detection/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [9]:
!pip install torch torchvision torchaudio pandas

In [12]:
!pip install xgboost==1.7.5


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 5.1 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.1.1
    Uninstalling xgboost-2.1.1:
      Successfully uninstalled xgboost-2.1.1


In [10]:
import torch
import pandas as pd

In [11]:
df = pd.read_csv('/content/drive/MyDrive/Data/dataset_diabetes.csv')

df.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,5.0,30.0,0.0,1.0,4.0,6.0,8.0
1,0.0,1.0,1.0,1.0,26.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,6.0,8.0
2,0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,10.0,0.0,1.0,13.0,6.0,8.0
3,0.0,1.0,1.0,1.0,28.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,3.0,0.0,3.0,0.0,1.0,11.0,6.0,8.0
4,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0,5.0,8.0


In [18]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score

In [19]:
y= df['Diabetes_binary']

X=df.drop('Diabetes_binary',axis=1)
# y.head()

# X.head()


In [20]:
# Create a StratifiedKFold object
n_splits = 5  # Number of folds
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize a list to store the accuracy scores for each fold
accuracy_scores = []

In [23]:


# Iterate through the folds
for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Set parameters for the XGBoost model
    params = {
      'objective': 'binary:logistic',
      'eval_metric': 'logloss',
      'eta': 0.1,
      'max_depth': 3
    }

    # Train the model
    model = xgb.train(params, dtrain, num_boost_round=100)

    # Make predictions on the test set
    y_pred = model.predict(dtest)

    # Evaluate the predictions and store the accuracy score
    accuracy = accuracy_score(y_test, y_pred.round())
    accuracy_scores.append(accuracy)

    print(f"Fold {fold + 1}: Accuracy = {accuracy}")



Fold 1: Accuracy = 0.7524577410000707
Fold 2: Accuracy = 0.7523162882806422
Fold 3: Accuracy = 0.7518036497382939
Fold 4: Accuracy = 0.7514499929268638
Fold 5: Accuracy = 0.7523695006365823


In [24]:
# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print(f"Average Accuracy: {average_accuracy}")

Average Accuracy: 0.7501839065445155


In [25]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

# ... (data loading and preprocessing) ...

# Create a StratifiedKFold object
n_splits = 5  # Number of folds
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store accuracy scores for each fold and each model
xgb_accuracy_scores = []
rf_accuracy_scores = []

# Iterate through the folds
for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # XGBoost model training and evaluation
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    params = {
        # ... (your XGBoost parameters) ...
    }
    xgb_model = xgb.train(params, dtrain, num_boost_round=100)
    xgb_y_pred = xgb_model.predict(dtest)
    xgb_accuracy = accuracy_score(y_test, xgb_y_pred.round())
    xgb_accuracy_scores.append(xgb_accuracy)

    # Random Forest model training and evaluation
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # Adjust parameters as needed
    rf_model.fit(X_train, y_train)
    rf_y_pred = rf_model.predict(X_test)
    rf_accuracy = accuracy_score(y_test, rf_y_pred)
    rf_accuracy_scores.append(rf_accuracy)

    print(f"Fold {fold + 1}: XGBoost Accuracy = {xgb_accuracy}, Random Forest Accuracy = {rf_accuracy}")

# Calculate average accuracy for each model
average_xgb_accuracy = sum(xgb_accuracy_scores) / len(xgb_accuracy_scores)
average_rf_accuracy = sum(rf_accuracy_scores) / len(rf_accuracy_scores)

print(f"Average XGBoost Accuracy: {average_xgb_accuracy}")
print(f"Average Random Forest Accuracy: {average_rf_accuracy}")

Fold 1: XGBoost Accuracy = 0.7463045477049296, Random Forest Accuracy = 0.7394440908126458
Fold 2: XGBoost Accuracy = 0.7482141594172148, Random Forest Accuracy = 0.7376759318197892
Fold 3: XGBoost Accuracy = 0.7502475597680012, Random Forest Accuracy = 0.7380110340925167
Fold 4: XGBoost Accuracy = 0.7468524543782713, Random Forest Accuracy = 0.7307257037770547
Fold 5: XGBoost Accuracy = 0.7498231715942849, Random Forest Accuracy = 0.7390720045268072
Average XGBoost Accuracy: 0.7482883785725404
Average Random Forest Accuracy: 0.7369857530057626


In [26]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

# ... (data loading and preprocessing) ...

# Create a StratifiedKFold object
n_splits = 5  # Number of folds
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store accuracy scores for each fold and each model
xgb_accuracy_scores = []
rf_accuracy_scores = []
lgb_accuracy_scores = []

# Iterate through the folds
for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # XGBoost model training and evaluation
    # ... (XGBoost code as before) ...

    # Random Forest model training and evaluation
    # ... (Random Forest code as before) ...

    # LightGBM model training and evaluation
    lgb_train = lgb.Dataset(X_train, label=y_train)
    lgb_test = lgb.Dataset(X_test, label=y_test, reference=lgb_train)
    params = {
        'objective': 'binary',  # Adjust objective as needed
        'metric': 'binary_logloss',  # Adjust metric as needed
        # ... (other LightGBM parameters) ...
    }
    lgb_model = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_test], num_boost_round=100)
    lgb_y_pred = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)
    lgb_accuracy = accuracy_score(y_test, (lgb_y_pred > 0.5).astype(int))  # Assuming binary classification
    lgb_accuracy_scores.append(lgb_accuracy)

    print(f"Fold {fold + 1}: XGBoost Accuracy = {xgb_accuracy}, Random Forest Accuracy = {rf_accuracy}, LightGBM Accuracy = {lgb_accuracy}")



/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


[LightGBM] [Info] Number of positive: 28277, number of negative: 28276
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016168 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 198
[LightGBM] [Info] Number of data points in the train set: 56553, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500009 -> initscore=0.000035
[LightGBM] [Info] Start training from score 0.000035
Fold 1: XGBoost Accuracy = 0.7498231715942849, Random Forest Accuracy = 0.7390720045268072, LightGBM Accuracy = 0.7513261192446424
[LightGBM] [Info] Number of positive: 28276, number of negative: 28277
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011228 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info]

In [28]:


# Calculate average accuracy for each model
# average_xgb_accuracy = sum(xgb_accuracy_scores) / len(xgb_accuracy_scores)
# average_rf_accuracy = sum(rf_accuracy_scores) / len(rf_accuracy_scores)
average_lgb_accuracy = sum(lgb_accuracy_scores) / len(lgb_accuracy_scores)

# print(f"Average XGBoost Accuracy: {average_xgb_accuracy}")
# print(f"Average Random Forest Accuracy: {average_rf_accuracy}")
print(f"Average LightGBM Accuracy: {average_lgb_accuracy}")

Average LightGBM Accuracy: 0.7524613978800361


In [29]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.svm import SVC  # Import SVC

# ... (data loading and preprocessing) ...

# Create a StratifiedKFold object
n_splits = 5  # Number of folds
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store accuracy scores for each fold and each model
xgb_accuracy_scores = []
rf_accuracy_scores = []
lgb_accuracy_scores = []
svm_accuracy_scores = []  # List for SVM accuracy scores

# Iterate through the folds
for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # XGBoost model training and evaluation
    # ... (XGBoost code as before) ...

    # Random Forest model training and evaluation
    # ... (Random Forest code as before) ...

    # LightGBM model training and evaluation
    # ... (LightGBM code as before) ...

    # SVM model training and evaluation
    svm_model = SVC(kernel='linear', random_state=42)  # Adjust kernel and other parameters as needed
    svm_model.fit(X_train, y_train)
    svm_y_pred = svm_model.predict(X_test)
    svm_accuracy = accuracy_score(y_test, svm_y_pred)
    svm_accuracy_scores.append(svm_accuracy)  # Append SVM accuracy to the list

    print(f"Fold {fold + 1}: XGBoost Accuracy = {xgb_accuracy}, Random Forest Accuracy = {rf_accuracy}, LightGBM Accuracy = {lgb_accuracy}, SVM Accuracy = {svm_accuracy}")

# Calculate average accuracy for each model
# ... (average accuracy calculations for XGBoost, Random Forest, LightGBM as before) ...


Fold 1: XGBoost Accuracy = 0.7498231715942849, Random Forest Accuracy = 0.7390720045268072, LightGBM Accuracy = 0.7548450983165936, SVM Accuracy = 0.7515382983237853
Fold 2: XGBoost Accuracy = 0.7498231715942849, Random Forest Accuracy = 0.7390720045268072, LightGBM Accuracy = 0.7548450983165936, SVM Accuracy = 0.7479312539783577
Fold 3: XGBoost Accuracy = 0.7498231715942849, Random Forest Accuracy = 0.7390720045268072, LightGBM Accuracy = 0.7548450983165936, SVM Accuracy = 0.7469939171028434
Fold 4: XGBoost Accuracy = 0.7498231715942849, Random Forest Accuracy = 0.7390720045268072, LightGBM Accuracy = 0.7548450983165936, SVM Accuracy = 0.7449427075965483
Fold 5: XGBoost Accuracy = 0.7498231715942849, Random Forest Accuracy = 0.7390720045268072, LightGBM Accuracy = 0.7548450983165936, SVM Accuracy = 0.749893902956571
Average SVM Accuracy: 0.7482600159916212


In [30]:
average_svm_accuracy = sum(svm_accuracy_scores) / len(svm_accuracy_scores)  # Calculate average SVM accuracy
print(f"Average SVM Accuracy: {average_svm_accuracy}")

Average SVM Accuracy: 0.7482600159916212


In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import xgboost as xgb
import lightgbm as lgb
import numpy as np

# Initialize the meta-model (Logistic Regression in this case)
meta_model = LogisticRegression()

# Create StratifiedKFold
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store accuracy scores and predictions
xgb_accuracy_scores = []
rf_accuracy_scores = []
lgb_accuracy_scores = []
svm_accuracy_scores = []
meta_accuracy_scores = []

# Meta-model training data
meta_train = np.zeros((len(X), 4))  # 4 because we have 4 models
meta_test = np.zeros((len(X_test), 4))

# Iterate through the folds
for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # SVM model
    svm_model = SVC(probability=True)
    svm_model.fit(X_train, y_train)
    svm_y_pred = svm_model.predict(X_test)
    svm_accuracy = accuracy_score(y_test, svm_y_pred)
    svm_accuracy_scores.append(svm_accuracy)

    # XGBoost model
    xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    xgb_model.fit(X_train, y_train)
    xgb_y_pred = xgb_model.predict(X_test)
    xgb_accuracy = accuracy_score(y_test, xgb_y_pred)
    xgb_accuracy_scores.append(xgb_accuracy)

    # Random Forest model
    rf_model = RandomForestClassifier(n_estimators=100)
    rf_model.fit(X_train, y_train)
    rf_y_pred = rf_model.predict(X_test)
    rf_accuracy = accuracy_score(y_test, rf_y_pred)
    rf_accuracy_scores.append(rf_accuracy)

    # LightGBM model
    lgb_train = lgb.Dataset(X_train, label=y_train)
    lgb_model = lgb.train(
        {'objective': 'binary', 'metric': 'binary_logloss'},
        lgb_train,
        num_boost_round=100
    )
    lgb_y_pred = (lgb_model.predict(X_test) > 0.5).astype(int)
    lgb_accuracy = accuracy_score(y_test, lgb_y_pred)
    lgb_accuracy_scores.append(lgb_accuracy)

    # Collect predictions for meta-model
    meta_train[test_index, 0] = svm_model.predict_proba(X_test)[:, 1]  # SVM probabilities
    meta_train[test_index, 1] = xgb_model.predict_proba(X_test)[:, 1]  # XGBoost probabilities
    meta_train[test_index, 2] = rf_model.predict_proba(X_test)[:, 1]   # Random Forest probabilities
    meta_train[test_index, 3] = lgb_model.predict(X_test)              # LightGBM probabilities

# Train meta-model (Logistic Regression) on the stacked predictions
meta_model.fit(meta_train, y)

# Meta-model evaluation on test data
meta_test[:, 0] = svm_model.predict_proba(X_test)[:, 1]
meta_test[:, 1] = xgb_model.predict_proba(X_test)[:, 1]
meta_test[:, 2] = rf_model.predict_proba(X_test)[:, 1]
meta_test[:, 3] = lgb_model.predict(X_test)

meta_y_pred = meta_model.predict(meta_test)
meta_accuracy = accuracy_score(y_test, meta_y_pred)
meta_accuracy_scores.append(meta_accuracy)

print(f"Meta-model accuracy: {meta_accuracy}")


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[LightGBM] [Info] Number of positive: 28277, number of negative: 28276
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015590 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 198
[LightGBM] [Info] Number of data points in the train set: 56553, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500009 -> initscore=0.000035
[LightGBM] [Info] Start training from score 0.000035


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[LightGBM] [Info] Number of positive: 28276, number of negative: 28277
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012109 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 197
[LightGBM] [Info] Number of data points in the train set: 56553, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499991 -> initscore=-0.000035
[LightGBM] [Info] Start training from score -0.000035


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[LightGBM] [Info] Number of positive: 28277, number of negative: 28277
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011850 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 199
[LightGBM] [Info] Number of data points in the train set: 56554, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[LightGBM] [Info] Number of positive: 28277, number of negative: 28277
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012275 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 198
[LightGBM] [Info] Number of data points in the train set: 56554, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


In [1]:
# Train meta-model (Logistic Regression) on the stacked predictions
meta_model.fit(meta_train, y)

# Meta-model evaluation on test data
meta_test[:, 0] = svm_model.predict_proba(X_test)[:, 1]
meta_test[:, 1] = xgb_model.predict_proba(X_test)[:, 1]
meta_test[:, 2] = rf_model.predict_proba(X_test)[:, 1]
meta_test[:, 3] = lgb_model.predict(X_test)

meta_y_pred = meta_model.predict(meta_test)
meta_accuracy = accuracy_score(y_test, meta_y_pred)
meta_accuracy_scores.append(meta_accuracy)

print(f"Meta-model accuracy: {meta_accuracy}")


NameError: name 'meta_model' is not defined